## Import libraries

In [1]:
# reload(sys)
import pandas as pd
import numpy as np
import os
import random
from pdb import set_trace as bp
import matplotlib.pyplot as plt
from collections import defaultdict
import pickle
import sys 
from sklearn.utils import shuffle
from dataGenerator import DataGenerator
from models import QuoraModel
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence
from utils import plotGraph,plotLengthHistogram,plotHist,labelPlot
from utils import createTiles, plotHistory

Using TensorFlow backend.


In [48]:
from keras.models import Sequential
from keras.layers import Input,Dense, LSTM,TimeDistributed, Flatten, Reshape
from keras.models import Model
import keras.backend as K
from keras.preprocessing import sequence

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization



class QuoraModel:
    
    def __init__(self, timeSlice = 2, questionLen = 25, wordEmbeddDim = 300, dataPath = './data', weightFile = 'model.h5' ):
        self.timeSlice = timeSlice
        self.questionLen = questionLen
        self.wordEmbeddDim = wordEmbeddDim
        self.weightFilePath = os.path.join(dataPath, weightFile )
        
    def getModel(self):
        layer1Dim = 500
        layer2Dim = 250
        nDense1   = 100
        inputs = Input(shape=(self.timeSlice,self.questionLen, self.wordEmbeddDim))
        lstm1 = TimeDistributed(LSTM(layer1Dim, activation = 'relu', return_sequences = False), name = 'lstm_1')(inputs)
#         lstm2 = TimeDistributed(LSTM(layer2Dim, activation = 'relu', return_sequences = False), name = 'lstm_2')(lstm1)
        flatten = Flatten()(lstm1)
#         dense1 = Dense(nDense1, activation = 'relu')(flatten)
        output = Dense(1, activation = 'sigmoid')(flatten)
        self.model = Model(inputs = inputs, outputs = output)
        if os.path.isfile(self.weightFilePath) :
            try:
                self.model.load_weights(self.weightFilePath)
                print("Weights loaded")
            except:
                pass
        return self.model
    
    def getModel2(self):
        layer1Dim = 500
        layer2Dim = 250
        num_dense   = 100
        rate_drop_lstm = 0.15 + np.random.rand() * 0.25
        rate_drop_dense = 0.15 + np.random.rand() * 0.25
        act = 'relu'
        bp()
        lstm_layer = LSTM(layer1Dim)

        sequence_1_input = Input(shape=(self.questionLen, self.wordEmbeddDim), dtype='int32')
        x1 = lstm_layer(sequence_1_input)
        
        sequence_2_input = Input(shape=(self.questionLen, self.wordEmbeddDim), dtype='int32')
        x2 = lstm_layer(sequence_2_input)

        merged = concatenate([x1, x2])
        merged = Dropout(rate_drop_dense)(merged)
        merged = BatchNormalization()(merged)
        merged = Dense(num_dense, activation=act)(merged)
        merged = Dropout(rate_drop_dense)(merged)
        merged = BatchNormalization()(merged)

        preds = Dense(1, activation='sigmoid')(merged)
        self.model = Model(inputs=[sequence_1_input, sequence_2_input], \
                           outputs=preds)
        
        return self.model
    
#     @staticmethod
#     def saveWeights(self):
#         try:
#             self.model.save_weights(self.weightFilePath)
#             print("Weights successfully saved")
#         except:
#             print("Exception: Weight save failed")

In [49]:
model = QuoraModel(questionLen = questionLen,
                   wordEmbeddDim = wordEmbeddDim).getModel2()

> <ipython-input-48-df04f3725f9c>(51)getModel2()
-> lstm_layer = LSTM(layer1Dim, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)
(Pdb) n
> <ipython-input-48-df04f3725f9c>(53)getModel2()
-> sequence_1_input = Input(shape=(self.questionLen, self.wordEmbeddDim), dtype='int32')
(Pdb) lstm_ayer
*** NameError: name 'lstm_ayer' is not defined
(Pdb) lstm_layer
(Pdb) n
> <ipython-input-48-df04f3725f9c>(54)getModel2()
-> x1 = lstm_layer(sequence_1_input)
(Pdb) sequence_1_input
<tf.Tensor 'input_19:0' shape=(?, 40, 100) dtype=int32>
(Pdb) n
ValueError: ValueErr...lizer.',)
> <ipython-input-48-df04f3725f9c>(54)getModel2()
-> x1 = lstm_layer(sequence_1_input)
(Pdb) lstm_layer.input
*** AttributeError: Layer lstm_19 is not connected, no input to return.
(Pdb) lstm_layer.shape
*** AttributeError: 'LSTM' object has no attribute 'shape'
(Pdb) lstm_layer = LSTM(layer1Dim)
(Pdb)  x1 = lstm_layer(sequence_1_input)
*** ValueError: Initializer for variable lstm_20/kernel/ is from inside a control-

BdbQuit: 

In [37]:
## Define parameters
timeSlice = 2
questionLen = 40
wordEmbeddDim = 100
batchSize  = 1024
nEpochs = 10
iterSize = 100

In [38]:
dataGenerator = DataGenerator(batchSize = batchSize, 
                              questionLen = questionLen,
                              wordEmbeddDim = wordEmbeddDim)
nData = dataGenerator.nTrainData

Data Generator succesfully initialized :) :) :) 


In [47]:
# This returns a tensor
model = QuoraModel(questionLen = questionLen,
                   wordEmbeddDim = wordEmbeddDim).getModel2()
model.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

TypeError: unsupported operand type(s) for *: 'Tensor' and 'float'

# train Network

In [42]:
trainHist = [[], []]
validHist = [[], []]
model2 = True
nIter = dataGenerator.nTrainData/(batchSize)
for i in range(nEpochs):
    for j in range(nIter):
        trainX, trainY = dataGenerator.getTrainBatch()
        if model2:
            trainX = [trainX[:,0], trainX[:,1]]
        model.fit( trainX, trainY, verbose = False )
        if( j%iterSize == 0 ):
            trainX, trainY = dataGenerator.getTrainBatch(batchSize = 5*batchSize)
            validX, validY = dataGenerator.getValidBatch(batchSize = 5*batchSize)
            if model2:
                trainX = [trainX[:,0], trainX[:,1]]
                validX = [validX[:,0], validX[:,1]]
            trainLoss, trainAcc = model.evaluate( trainX, trainY, verbose = False )
            validLoss, validAcc = model.evaluate(validX, validY, verbose = False )
            print( "Epoch:{0}\t Iter: {1}\t TrainLoss {2:.4f}\t ValidLos: {3:.4f} \
            TrainAcc: {4:.4f}\t ValidAcc: {5:.4f}"
                   .format(i, j+1, trainLoss, validLoss, trainAcc, validAcc))
            trainHist[0].append(trainLoss)
            trainHist[1].append(trainAcc)
            validHist[0].append(validLoss)
            validHist[1].append(validAcc)
    model.save_weights('data/model.h5')
plotHistory(trainHist, validHist)

NameError: name 'false' is not defined

In [20]:
# dirpath = './data'
# embeddFileName = 'embeddDict_glove_6b_100d.pickle'
# with open(os.path.join(dirpath, embeddFileName), 'rb') as handle:
#     embeddDict = pickle.load(handle)

In [23]:
QuoraModel.saveWeights()

Exception: Weight save failed
